In [13]:
import numpy as np
from sklearn import svm

In [20]:
train = np.loadtxt(open("data/train.csv", "r"), delimiter=",", skiprows=1)
test = np.loadtxt(open("data/test.csv", "r"), delimiter=",", skiprows=1)

In [22]:
trainResults = train[:,0]
trainData = train[:,1:]

In [ ]:
clf = svm.SVC()   
classifier = clf.fit(trainData, trainResults)

In [ ]:
classifier.predict(test)